# **Data Cleaning Notebook**

## Objectives

* Evaluate missing data
* Clean the data
* Split data into train and test sets

## Inputs

* outputs/datasets/collection/house_prices_records.csv
* outputs/datasets/collection/inherited_houses.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Collected Data

Load the collected house price record data

In [ ]:
import pandas as pd
df_raw_path = "outputs/datasets/collection/house_prices_records.csv"
df = pd.read_csv(df_raw_path)
df.head(5)

# Missing Data

Review missing data

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from ydata_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

We plot the correlation heatmaps and predictive power scores between all the variables

Custom function from Code Institute Walkthrough Project 02

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    """ To display correlation heatmaps """
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='plasma',
                    annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    """ To display PPS heatmap """
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r',
                         annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    """ To calculate Spearman and Pearson correlations and PPS score """
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(
        columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query(
        "ppscore < 1").filter(['ppscore']).describe().T
    print(
        "PPS threshold - check PPS score IQR to decide threshold for heatmap\n"
         )
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman,
                      pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):
    """ To display correlation and PPS heatmaps """
    print("\n")
    print("* Analyse how the target variable for your ML models are "
          "correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are "
          "correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold,
                 figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous "
          "variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold,
                 figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two "
          f"columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect "
          f"predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize,
                font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

In [ ]:
DisplayCorrAndPPS(df_corr_pearson=df_corr_pearson,
                  df_corr_spearman=df_corr_spearman,
                  pps_matrix=pps_matrix,
                  CorrThreshold=0.4, PPS_Threshold=0.2,
                  figsize=(12, 10), font_annot=10)

---

# Handling Missing Data

Load data into a new variable

In [ ]:
# Load CSV data into a new variable to avoid accidental overwriting
df_missing = pd.read_csv(df_raw_path)

## 2ndFlrSF

- There are 86 missing data entries, equating to 5.9%
- Reasons for missing data:
    - There is a possibility that some houses do not have a second floor - 53.5% of data is a zero
    - Data may have not been inputed, due to user error

There are two other variables:
1. GrLivArea - which is the total above ground living area square feet
2. 1stFlrSF - which is the total first floor square feet

All three variables are in square feet. Therefore for any missing data, where the data was not entered due to error. We input the difference between the two variables GrLivArea and 1stFlrSF. Any negative values will be replaced with a zero. There were not missing data after this step, however if there were than the value of zero should be inputed


In [ ]:

# Display number of missing values in '2ndFlrSF
print("Initial missing data in '2ndFlrSF':\n", df_missing['2ndFlrSF'].isna().sum())

# Replace missing values in '2ndFlrSF'
df_missing['2ndFlrSF'] = df.apply(
    lambda row: row['GrLivArea'] - row['1stFlrSF'] if pd.isna(row['2ndFlrSF']) else row['2ndFlrSF'], axis=1
)

# Display number of missing values in '2ndFlrSF
print("Updated missing data in '2ndFlrSF':\n", df_missing['2ndFlrSF'].isna().sum())


## BedroomAbvGr

- There are 99 missing data entries, equating to 6.8%
- Reasons for missing data:
    - There is a possibility that some houses do not have a bedrooms above ground
    - Data may have not been inputed, due to user error

Given there is a low number of zeros, we believe that it is unlikely that there are no bedrooms above ground. Therfore we have replaced all missing values with the median number of 3 bedrooms.

In [ ]:
# Display number of missing values in 'BedroomAbvGr'
print("Initial missing data in 'BedroomAbvGr':\n", df_missing['BedroomAbvGr'].isna().sum())

# Replace missing values with '3'
df_missing['BedroomAbvGr'] = df['BedroomAbvGr'].fillna(3)

# Display number of missing values in 'BedroomAbvGr'
print("Updated missing data in 'BedroomAbvGr':\n", df_missing['BedroomAbvGr'].isna().sum())

## BsmtFinType1

'' = IF TotalBsmtSF > 0, then Unf, ELSE None

- There are 114 missing data entries, equating to 7.8%
- Reasons for missing data:
    - Some houses do not have basements and there was an error in inputting the correct value or decision to purposefully leave as blank.
    - There is an error in putting in the value despite there being a basement

If there is a basement identified by TotalBsmtSF > 0 (i.e. there is document squrae foot of basement), we will replace the missing data with 1 ('Unfinished'). If there is no document squae foot of basement then we will replace missing data with 0 ('No basement')


In [ ]:
# Display number of missing values in 'BsmtFinType1'
print("Initial missing data in 'BsmtFinType1':\n", df_missing['BsmtFinType1'].isna().sum())

# Replace missing values in 'BsmtFinType1'
df_missing['BsmtFinType1'] = df.apply(
    lambda row: 1 if pd.isna(row['BsmtFinType1']) and row['TotalBsmtSF'] > 0 else (0 if pd.isna(row['BsmtFinType1']) else row['BsmtFinType1']), axis=1
)

# Display number of missing values in 'BsmtFinType1'
print("Updated missing data in 'BsmtFinType1':\n", df_missing['BsmtFinType1'].isna().sum())

## EnclosedPorch

- There are 1324 missing data entries, equating to 90.7%
- Reasons for missing data:
    - There is a possibility that the majority of houses do not have an enclosed porch
    - Data may have not been inputed, due to user error

There is another variable OpenPorchSF, which also has a higher number of zero but no missing data.
There are no observered correlations between this variable and others as well as no stong correlation to SalePrice.

As a result due to the large volume of missing data, this variable has been dropped from the data set.

In [ ]:
# Display number of missing values in 'EnclosedPorch'
print("Initial missing data in 'EnclosedPorch':\n", df_missing['EnclosedPorch'].isna().sum())

# Drop column from dataset
df_missing.drop(columns=['EnclosedPorch'], inplace=True)

# Check column no longer exists
df_missing.info()

## GarageFinish

'GarageFinish' = IF GarageArea > 0, then Unf, Else None
- There are 162 missing data entries, equating to 11.1%
- Reasons for missing data:
    - There is a possibility that some houses do not have a garage, therefore this data was not entered
    - Data may have been missed despite a garage being present

There is another varibale that provides GarageArea which has no missing data. We will apply similar logic to previous varibles where if the GarageArea > 0 then we will replace missing data to '1' (Unfinished). If there is no garage (i.e. GarageArea <0), we will replace missing data with "0" (no garage)



In [ ]:
# Display number of missing values in 'GarageFinish
print("Initial missing data in 'GarageFinish':\n", df_missing['GarageFinish'].isna().sum())

# Replace missing values in 'GarageFinish'
df_missing['GarageFinish'] = df.apply(
    lambda row: 1 if pd.isna(row['GarageFinish']) and row['GarageArea'] > 0 else (0 if pd.isna(row['GarageFinish']) else row['GarageFinish']), axis=1
)

# Display number of missing values in 'GarageFinish
print("Updated missing data in 'GarageFinish':\n", df_missing['GarageFinish'].isna().sum())

## GarageYrBlt

'GarageYrBlt' = IF GarageFinish == None (i.e. no garage), then .... , ELSE YearBuilt
- There are 81 missing data entries, equating to 5.5%
- Reasons for missing data:
    - We know some houses do not have garages, therefore this data may have intentionally been missed
    - Data may have not been inputed, due to user error

There is a strong correlation between the house year build (YearBuilt) and the garage year build (GarageYrBlt). Spearman and Pearson analysis both demonstrate strong correlations with figures of 0.89 and 0.83. The YearBuilt variable has a high PPS score of 0.7 for GarageYrBlt.

Given the above, using similar logic to above, if there is a garage present then we will replace any missing values with the YearBuilt. If no garage is present then we will replace with value of '0' This will need to be taken into consideration in future steps.

In [ ]:
# Display number of missing values in 'GarageYrBlt'
print("Initial missing data in 'GarageYrBlt':\n", df_missing['GarageYrBlt'].isna().sum())

# Replace missing values in 'GarageYrBlt'
df_missing['GarageYrBlt'] = df.apply(
    lambda row: '0' if pd.isna(row['GarageYrBlt']) and row['GarageArea'] < 0 else (row['YearBuilt'] if pd.isna(row['GarageYrBlt']) else row['GarageYrBlt']), axis=1
)

# Display number of missing values in 'GarageFinish
print("Updated missing data in 'GarageYrBlt':\n", df_missing['GarageYrBlt'].isna().sum())

## LotFrontage

'LotFrontage' = Assume none is missing, therefore 0
- There are 259 missing data entries, equating to 17.7%
- Reasons for missing data:
    - It is likely that this data was missed in error

Lot frontage is the length of the lot that is facing a street. Assuming that a pot is equal in length and width, then the lot frontage will be the square root of the lot area. I.e. if a lot is 100ft x 100ft then the LotFrontage is 100ft and the LotArea is 10000 squared foot.

We know there is no missing data from the LotArea variable, however we cannot make the assumption that all plots of land are square. The data for the LotFrontage appears normally distributed (ignoring the missing data points).

Therefore, we will replace all missing data with the mean LotFrontage value of 70

In [ ]:
# Display number of missing values in 'LotFrontage'
print("Initial missing data in 'LotFrontage':\n", df_missing['LotFrontage'].isna().sum())

# Replace missing values with '70'
df_missing['LotFrontage'] = df['LotFrontage'].fillna(70)

# Display number of missing values in 'LotFrontage'
print("Updated missing data in 'LotFrontage':\n", df_missing['LotFrontage'].isna().sum())

## MasVnrArea

- There are 8 missing data entries, equating to 0.5%
- Reasons for missing data:
    - Likely missed due to error

Given there is a large number of zeros, suggesting that many of the houses do not have this features. We will assume the missinf data has been missed in error. We will replace all missing values with '0'.

In [ ]:
# Display number of missing values in 'MasVnrArea'
print("Initial missing data in 'MasVnrArea':\n", df_missing['MasVnrArea'].isna().sum())

# Replace missing values with '0'
df_missing['MasVnrArea'] = df['MasVnrArea'].fillna(0)

# Display number of missing values in 'MasVnrArea'
print("Updated missing data in 'MasVnrArea':\n", df_missing['MasVnrArea'].isna().sum())

## WoodDeckSF

- There are 1305 missing data entries, equating to 89.4%
- Reasons for missing data:
    - There is a possibility that some houses do not have any wood decking
    - Data may have not been inputed, due to user error

There is a large number of missing data. There are no clear correlations between WooDeckSF and SalePrice. There are not clear correlations between WooDeckSf and other variables. As a result we will drop this variable from the dataset.

In [ ]:
# Display number of missing values in 'WoodDeckSF'
print("Initial missing data in 'WoodDeckSF':\n", df_missing['WoodDeckSF'].isna().sum())

# Drop column from dataset
df_missing.drop(columns=['WoodDeckSF'], inplace=True)

# Check column no longer exists
df_missing.info()

---

# Review Data Types

Following the above data review, we have removed 2 variables and their columns of data. We have also replaced all missing data.

We can see that we have float, integer and object types.
On review:
- There does not seem to be any float type entries (i.e. no decimal points).
- Object types, after replacing missing, are appear to be integers


In [ ]:
df_missing.head(5)

In [ ]:
df_missing.info()

We use the following code to confirm that all float types are in fact integers and convert these to integers

In [ ]:
# Using the .is_integer function we check each float type
def check_integer(df):
    cols_to_check = ['2ndFlrSF','BedroomAbvGr','BsmtFinType1','GarageFinish','GarageYrBlt','LotFrontage','MasVnrArea',]

    for col in cols_to_check:
        if df[col].apply(float.is_integer).all():
            return f"All floats are listed as integers"
        else:
            return "There are some floats listed as floats"

check_integer(df_missing)

In [ ]:
variables_as_float_type = ['2ndFlrSF','BedroomAbvGr','BsmtFinType1','GarageFinish','GarageYrBlt','LotFrontage','MasVnrArea',]

df_missing[variables_as_float_type] = df_missing[variables_as_float_type].astype(int)

In [ ]:
df_missing.info()

---

# Missing Data Recheck

In [ ]:
df_missing.head(10)

In [ ]:
check_missing_data = df_missing.columns[df_missing.isna().sum() > 0].to_list()

from ydata_profiling import ProfileReport
if check_missing_data:
    profile = ProfileReport(df=df_missing, minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

---

# Split Test and Train Dataset

In [ ]:
from sklearn.model_selection import train_test_split

df = df_missing

TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

Evaluate missing data function taken from Code Institute walkthrough project 2 to confirm there are no missing data in the train set.

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

Evaluate missing data in inherited houses dataset after loading dataset

In [ ]:
df_raw_path_inherited = "outputs/datasets/collection/inherited_houses.csv"
df_inherited = pd.read_csv(df_raw_path_inherited)
df_inherited.head(5)

In [ ]:
df_missing_data_inherited = EvaluateMissingData(df_inherited)
print(f"* There are {df_missing_data_inherited.shape[0]} variables with missing data \n")
df_missing_data_inherited

---

# Push cleaned data to Repo

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

## Train Set

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

## Test Set

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)

---